In [4]:
import csv
import serial
import numpy as np
import urllib.request 

import scipy.signal
from sklearn.externals import joblib

In [ ]:
try:
    arduino = serial.Serial("/COM9",timeout=1)
    print("The port has been successfully loaded.")
except:
    print("Please check the port.")

In [ ]:
dataset = []
data_count = 0

while data_count<11:
    rawdata = []
    count = 0
    while count<100:
        rawdata.append(int(arduino.readline()))
        count+=1
    dataset.append(rawdata)
    data_count+=1

filtered_values = []

mean_value = np.mean(dataset[10])
value_25 = np.percentile(dataset[10], 25)
value_75 = np.percentile(dataset[10], 75)

for value in dataset[0]:
    filtered_values.append(value_75 if value > mean_value else value_25)
    
extracted_parameters = [np.amin(filtered_values), np.amax(filtered_values),
                        np.ptp(filtered_values), np.percentile(filtered_values, 75),
                        np.percentile(filtered_values, 25), np.median(filtered_values),
                        np.mean(filtered_values), round(np.std(filtered_values),2),
                        round(np.var(filtered_values),2), round(scipy.stats.kurtosis(filtered_values),2),
                        round(scipy.stats.skew(filtered_values),2)]

In [17]:
print('\nThe parameters of this signal are: ', extracted_parameters)

[490.0, 496.0, 6.0, 496.0, 496.0, 496.0, 495.22]


In [18]:
inputarray = extracted_parameters[:7]
clf = joblib.load('model.pkl')
prediction = clf.predict([inputarray])
percentage_predictions = clf.predict_proba([inputarray])

percentage_predictions = np.array(percentage_predictions.tolist())

print({'ceramics':percentage_predictions[0][0],
             'plastic':percentage_predictions[0][1],'wood':percentage_predictions[0][2]})

{'ceramics': 0.0390936227134829, 'plastic': 0.9253597476988202, 'wood': 0.03554662958769686}


In [ ]:
arduino.close()